#  Molecular Attention Transformer 的介绍

在本系列教程的前几节中，你已经学了如何用DeepChem在各种应用中训练模型。但我们还没有真正研究过模型的可解释性问题。

很多时候，在建模时我们会被问到这样的问题--这个模型是如何工作的？ 我们为什么要相信这个模型？ 作为一个数据科学家，我的回答通常是："因为我们已经严格证明了模型在测试集上的性能，而测试集的分割对于现实世界来说是真实的"。但是很多时候，这并不足以说服领域专家。

[LIME](https://homes.cs.washington.edu/~marcotcr/blog/lime/) 是一个可以帮助解决这个问题的工具。 它使用特征空间的局部扰动来确定特征的重要性。在本教程中，你将学习如何与DeepChem一起使用LIME来解释我们的模型正在学习什么。

![Selection_110.png](https://github.com/deepchem/deepchem/blob/master/examples/tutorials/assets/lime_dog.png?raw=1)

那么，如果这个工具能以人类可理解的方式对图像进行处理，它能在分子上发挥作用吗？ 在本教程中，你将学习如何使用LIME对任何一个固定长度的特征化模型进行模型的解释。

## Colab

本教程和本序列的其他内容都是在Google colab中完成的。如果你想在colab中打开这个笔记本，你可以使用以下链接。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Introduction_to_Model_Interpretability.ipynb)



#  模型可解释性介绍

In [7]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


'2.6.1.dev'

## 模型的生成

让我们首先加载具有ECFP(Extended Connectivity Fingerprint)特征化的Tox21数据集。回顾一下这种特征化的工作方式:它识别每个分子中的小片段，然后将输出向量的元素设置为1，以表示哪些片段存在于某个特定的分子中。

In [8]:
import deepchem as dc
n_features = 1024
tasks, datasets, transformers = dc.molnet.load_tox21(featurization='ecfp')
train_dataset, valid_dataset, test_dataset = datasets

现在让我们训练一个模型来处理这个数据集。和以前的教程一样，我们将使用一个多任务分类器(MultitaskClassifier)，它是一堆简单的密集层。

In [9]:
n_tasks = len(tasks)
n_features = train_dataset.get_data_shape()[0]
model = dc.models.MultitaskClassifier(n_tasks, n_features)
model.fit(train_dataset, nb_epoch=50)

AttributeError: module 'deepchem.models' has no attribute 'MultitaskClassifier'

我们在训练集和验证集上评估这个模型，以获得对其准确性的一些基本了解。我们将使用ROC-AUC作为我们选择的指标。

In [ ]:
import numpy as np
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
print("Train scores")
print(model.evaluate(train_dataset, [metric], transformers))
print("Validation scores")
print(model.evaluate(valid_dataset, [metric], transformers))

## 使用 LIME

该模型似乎在预测哪些分子有毒方面做得很合理，但它是如何工作的？ 当它预测一个特定的分子有毒或无毒时，分子的哪些方面导致了这种预测？ 这就是*可解释性*的本质：学习为什么一个输入会导致某种预测的发生。

LIME(Local Interpretable Model-Agnostic Explanations)是一个解决这一问题的工具。这个名字是 "局部可解释模型-不可知论解释 "的简称。 它可以在任何具有固定大小的输入向量的问题上工作。 它通过计算各个特征的概率分布和特征间的协方差来工作。 这使得它能够在样本附近构建一个局部线性模型，描述输入的局部扰动会对输出产生最大影响。 也就是说，在分子中添加或删除哪些片段最有可能改变有毒和无毒之间的预测结果？

首先，我们需要安装lime。幸运的是，lime在`pip`上很方便，所以你可以从这个Jupyter笔记本中安装它。

In [ ]:
!pip install lime

现在我们已经安装了lime，我们要为`lime`创建一个`Explainer`对象。这个对象将接收训练数据集和特征名称。我们使用ECFP作为我们的特征。我们的特征没自然的名字，所以我们只是对它们进行数字编号。另一方面，我们的标签也有自然的名字。回顾一下，Tox21是用于毒性检测的；所以我们把0称为 "无毒"，1称为 "有毒"。

In [ ]:
from lime import lime_tabular
feature_names = ["fp_%s"  % x for x in range(1024)]
explainer = lime_tabular.LimeTabularExplainer(train_dataset.X, 
                                              feature_names=feature_names, 
                                              categorical_features=feature_names,
                                              class_names=['not toxic', 'toxic'], 
                                              discretize_continuous=True)

我们将尝试解释为什么该模型预测一个分子对NR-AR有毒。
具体的检测细节可以在[链接](https://pubchem.ncbi.nlm.nih.gov/bioassay/743040)中找到。

In [ ]:
# We need a function which takes a 2d numpy array (samples, features) and returns predictions (samples,)
def eval_model(my_model):
    def eval_closure(x):
        ds = dc.data.NumpyDataset(x, n_tasks=12)
        # The 0th task is NR-AR
        predictions = my_model.predict(ds)[:,0]
        return predictions
    return eval_closure

model_fn = eval_model(model)

现在让我们尝试在一个特定的分子上使用这个评价函数。让我们在测试集中挑选第一个被正确预测为有毒的分子（也就是说，这个分子是有毒的，而且模型正确预测了它的毒性）。

In [ ]:
from rdkit import Chem
active_id = np.where((test_dataset.y[:,0] == 1) * (model.predict(test_dataset)[:,0,1] > 0.8))[0][0]
Chem.MolFromSmiles(test_dataset.ids[active_id])

现在我们有了一个训练好的模型和一个分子，让我们要求 "Explainer "弄清楚为什么这个分子被预测为有毒。 我们要求它提供预测最灵敏的100个特征（即指纹中的元素，每个元素对应一个或多个片段）。

In [ ]:
exp = explainer.explain_instance(test_dataset.X[active_id], model_fn, num_features=100, top_labels=1)

返回的值是一个'Explanation'对象。 它有一些方法，你可以调用这些方法来检索各种形式的结果。 一个方便的交互式工作形式是'show_in_notebook()'，提供一个图形化的表示。

In [ ]:
exp.show_in_notebook(show_table=True, show_all=False)

这个输出需要一些解释。 在左边，它显示这个分子被预测为有毒。 当然，我们已经知道这一点。 这就是我们选择它的原因。 在右边，它列出了指纹中对预测影响最大的100个元素。 对于每一个元素，其数值栏表示相应的片段在这个分子中是否存在（1.00）或不存在（0.00）。 而在中间，它显示了每个指数的值是否有助于预测为无毒（蓝色）或有毒（橙色）。

大部分的片段都不存在。 它告诉我们的是那些片段，*如果*它们存在的话，会改变预测结果。 我们对这些并不感兴趣。 我们想知道分子中存在的、对预测有贡献的片段。 让我们试着把这些结果放到一个更有用的形式中。

首先，指纹中的索引并不是很有信息量。 让我们写一个函数来逆转特征化，从索引映射回激活它们的片段。

In [ ]:
def fp_mol(mol, fp_length=1024):
    """
    returns: dict of <int:list of string>
        dictionary mapping fingerprint index
        to list of SMILES strings that activated that fingerprint
    """
    d = {}
    feat = dc.feat.CircularFingerprint(sparse=True, smiles=True, size=1024)
    retval = feat._featurize(mol)
    for k, v in retval.items():
        index = k % fp_length
        if index not in d:
            d[index] = set()
        d[index].add(v['smiles'])
    return d

# What fragments activated what fingerprints in our active molecule?
my_fragments = fp_mol(Chem.MolFromSmiles(test_dataset.ids[active_id]))

现在我们想查询`Explanation'，看看这些片段中哪些对预测有贡献。 我们可以使用`as_map()`方法，以更适合处理的形式获得信息。

In [ ]:
print(exp.as_map())

这个地图中的键是标签，我们只有一个。 值是一个图元的列表，每个图元的形式是（fingerprint_index, weight）。 让我们把它转换为一个将指数映射为权重的字典。

In [ ]:
fragment_weight = dict(exp.as_map()[1])

我们知道哪些片段存在于我们感兴趣的分子中（`my_fragments`），我们也知道哪些片段对预测有贡献（`fragment_weights`）。 让我们把它们循环起来并打印出来。

In [ ]:
for index in my_fragments:
    if index in fragment_weight:
        print(index, my_fragments[index], fragment_weight[index])

这些就是对预测主要起作用的分子片段。

# 恭喜！是时候加入社区了！

恭喜您完成本教程笔记本！如果您喜欢本教程并希望继续使用 DeepChem，我们鼓励您完成本系列的其余教程。您还可以通过以下方式帮助 DeepChem 社区：

## 在 [GitHub](https://github.com/deepchem/deepchem) 上为 DeepChem 点亮小星星
这有助于大家建立对 DeepChem 项目和我们正在尝试构建的开源药物发现工具的共识。

## 加入 DeepChem Gitter
DeepChem [Gitter](https://gitter.im/deepchem/Lobby) 聚集了许多对生命科学深度学习感兴趣的科学家、开发人员和爱好者，欢迎加入！